In [21]:
# install transformers
!pip install transformers --no-build-isolation
!pip install datasets

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [22]:
import os

os.getcwd()
os.listdir()

['test', 'multimodal']

In [23]:
import torch
print(torch.cuda.is_available())  # 應該回傳 True
print(torch.version.cuda)  # 應該顯示 CUDA 版本

True
12.6


In [24]:
from datasets import Dataset, Image
from datasets import load_dataset, Image
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
dataset = Dataset.from_dict({"image": ["/workspace/multimodal/Statue-of-Liberty-Island-New-York-Bay.png", "/workspace/multimodal/bee.png"]}).cast_column("image", Image())
image1 = dataset[0]["image"]
image2 = dataset[1]["image"]


In [25]:
from transformers import AutoProcessor, AutoModelForVision2Seq
processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
import torch
model = AutoModelForVision2Seq.from_pretrained(
    "HuggingFaceTB/SmolVLM-Instruct",
    torch_dtype=torch.bfloat16
).to("cuda")

In [26]:
# Create input messages
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "image"},
            {"type": "text", "text": "Can you describe the two images?"}
        ]
    },
]
# Prepare inputs
prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
inputs = processor(text=prompt, images=[image1, image2], return_tensors="pt")
inputs = inputs.to(DEVICE)
# Generate outputs
generated_ids = model.generate(**inputs, max_new_tokens=500)
generated_texts = processor.batch_decode(
    generated_ids,
    skip_special_tokens=True,
)
print(generated_texts[0])


User:<image>Can you describe the two images?
Assistant: The first image shows a green statue of the Statue of Liberty standing on a stone pedestal in front of a body of water. The statue is holding a torch in its right hand and is surrounded by a cityscape with tall buildings and skyscrapers. The water in the background is calm and there are trees and buildings on the shore. The second image shows a bee on a pink flower. The bee is black and yellow and is surrounded by pink petals.


In [27]:
"""
User:<image>Can you describe the two images?
Assistant: I can describe the first one, but I can't describe the second one.
"""

"\nUser:<image>Can you describe the two images?\nAssistant: I can describe the first one, but I can't describe the second one.\n"